In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 69 kB 6.8 MB/s 


In [ ]:
import seaborn as sns
import pandas as pd
data_url = '/content/drive/MyDrive/ai13/val_stats.csv'
df=pd.read_csv(data_url)

df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (0,10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


region           name     tag   rating  damage_round headshots  \
0    NaN       ShimmyXD    #NA1  Radiant         135.8       992   
1    NaN      XSET Cryo  #cells  Radiant         170.3       879   
2    NaN  PuRelittleone  #yoruW  Radiant         147.5       720   
3    NaN           Boba   #0068  Radiant         178.2       856   
4    NaN    i love mina  #kelly  Radiant         149.8       534   

   headshot_percent  aces  clutches  flawless  ... gun2_name gun2_head  \
0              24.9     0       140        80  ...   Phantom        33   
1              28.3     2       122        94  ...  Operator         8   
2              24.0     3       117        59  ...   Phantom        36   
3              37.3     3        83        49  ...   Sheriff        48   
4              24.4     2        71        38  ...   Spectre        21   

  gun2_body gun2_legs  gun2_kills  gun3_name  gun3_head  gun3_body  gun3_legs  \
0        62         5         220    Classic         36         60          3   
1        91         0         226    Phantom         32         63          5   
2        61         3         231   Operator          8         91          1   
3        51         1          48    Phantom         44         56          0   
4        71         8          65   Operator          8         92          0   

   gun3_kills  
0         147  
1         137  
2         102  
3          36  
4          64  

[5 rows x 38 columns]

In [ ]:
df.columns

Index(['region', 'name', 'tag', 'rating', 'damage_round', 'headshots',
       'headshot_percent', 'aces', 'clutches', 'flawless', 'first_bloods',
       'kills', 'deaths', 'assists', 'kd_ratio', 'kills_round', 'most_kills',
       'score_round', 'wins', 'win_percent', 'agent_1', 'agent_2', 'agent_3',
       'gun1_name', 'gun1_head', 'gun1_body', 'gun1_legs', 'gun1_kills',
       'gun2_name', 'gun2_head', 'gun2_body', 'gun2_legs', 'gun2_kills',
       'gun3_name', 'gun3_head', 'gun3_body', 'gun3_legs', 'gun3_kills'],
      dtype='object')

In [ ]:
df.shape # 38개의 열, 8만여개의 행..., 발로란트 상위권 플레이어들의 데이터.
# 머신러닝분석으로 회귀 ? 분류 ?........................... 타겟값은 어떤 것으로 . 무엇을 예측하는 ?.. 지표를 선택 해야한다..

(85678, 38)

In [ ]:
df.win_percent.mean() # 1. 승패예측이라고 가정할 때, 전체 플레이어의 승률 평균은 53% ..............
# 어떤영웅에 어떤무기를 갖고 ~ 의 승률을 낼것이다. 라는 모델을 학습할 때, 
# 파이프라인에 텍스트형으로 된 여러개의 영웅명들과 무기들을 ordinal로 변환하여 구별가능하긴 함...
# 다른 지표들도 많긴하다. 라운드별 데미지, 그리고 헤드샷 비율, 킬데스 비율, 등등............
# 하지만 추후 타겟값으로 예측할 때, 임의 에이전트, 무기를 골랐을 때 승률을 예측하는 모델이 가장 이상적이다.
# 따라서 모델 목표는 < 영웅과 무기의 선택으로 승률을 예측하는 모델 > 을 만들것이다.

53.87893041387522

In [ ]:
# 타겟을 만들자..  wintag, 0/1 (winparset 49 lows =0, 50 highs=1)
df['wintag']=df['win_percent'].apply(lambda x:1 if x >= 50 else 0 )

In [ ]:
df['wintag'].mean()

0.696269754195943

In [ ]:
df.columns #필요한 컬럼만 구하기.  [wintag, gun1_name, agent_1]

Index(['region', 'name', 'tag', 'rating', 'damage_round', 'headshots',
       'headshot_percent', 'aces', 'clutches', 'flawless', 'first_bloods',
       'kills', 'deaths', 'assists', 'kd_ratio', 'kills_round', 'most_kills',
       'score_round', 'wins', 'win_percent', 'agent_1', 'agent_2', 'agent_3',
       'gun1_name', 'gun1_head', 'gun1_body', 'gun1_legs', 'gun1_kills',
       'gun2_name', 'gun2_head', 'gun2_body', 'gun2_legs', 'gun2_kills',
       'gun3_name', 'gun3_head', 'gun3_body', 'gun3_legs', 'gun3_kills',
       'wintag'],
      dtype='object')

In [ ]:
valdf=df[['agent_1','gun1_name','wintag','damage_round','kills_round','kd_ratio','clutches']]
valdf.shape

(85678, 7)

In [ ]:
valdf.head()

agent_1 gun1_name  wintag  damage_round  kills_round  kd_ratio  clutches
0     Fade    Vandal       1         135.8          0.7      1.07       140
1  Chamber    Vandal       1         170.3          1.0      1.35       122
2     Yoru    Vandal       1         147.5          0.8      1.05       117
3     Jett    Vandal       1         178.2          1.0      1.40        83
4     Jett    Vandal       1         149.8          0.8      1.11        71

In [ ]:
from sklearn.model_selection import train_test_split

target = 'wintag'
train, test = train_test_split(valdf, random_state=42, train_size=0.8)
train, val = train_test_split(train, random_state=2,train_size=0.8)
print("train shape: ", train.shape)
print("test shape: ", test.shape)
print("val shape: ", val.shape)

train shape:  (54833, 7)
test shape:  (17136, 7)
val shape:  (13709, 7)


In [ ]:
features = train.drop(columns=[target]).columns

X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]

print(X_train.shape, X_val.shape, X_test.shape)

(54833, 6) (13709, 6) (17136, 6)


In [ ]:
# 타겟 데이터 범주
y_train = train[target]
y_train.value_counts(normalize=True)

1    0.694746
0    0.305254
Name: wintag, dtype: float64

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.countplot(x=y_train);

In [ ]:
# 기준모델
major = y_train.mode()[0]
# 타겟 샘플 수 만큼 기준모델로 예측
y_pred = [major] * len(y_train)
# 최다 클래스의 빈도
from sklearn.metrics import accuracy_score
print("training accuracy: ", accuracy_score(y_train, y_pred))
y_val = val[target]
y_pred = [major] * len(y_val)
print("validation accuracy: ", accuracy_score(y_val, y_pred))

training accuracy:  0.6947458647165028
validation accuracy:  0.6994675031001532


In [ ]:
from sklearn.pipeline import make_pipeline
from category_encoders import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# 파이프라인.
pipe = make_pipeline(
    OrdinalEncoder(), 
    SimpleImputer(), 
    RandomForestClassifier(n_estimators=50, random_state=4, n_jobs=-1,max_depth = 15)
)


pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_val)
print('검증 정확도: ', accuracy_score(y_val, y_pred))

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


검증 정확도:  0.705157196002626


In [ ]:
import pickle
with open('model.pkl','wb') as pickle_file:
  pickle.dump(pipe,pickle_file)

In [ ]:
print(pipe)

Pipeline(steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['agent_1', 'gun1_name'],
                                mapping=[{'col': 'agent_1',
                                          'data_type': dtype('O'),
                                          'mapping': Jett          1
Chamber       2
Sage          3
Reyna         4
Fade          5
Sova          6
Omen          7
Raze          8
Brimstone     9
Viper        10
Killjoy      11
KAY/O        12
Yoru         13
Skye         14
Breach       15
Astra        16
Phoenix      17
Neon         18
Cypher       19
NaN          -2
dtype: int64},
                                         {'col': 'gun1_name',
                                          'data_type': dtype('O'),
                                          'mapping': Vandal       1
Odin         2
Phantom      3
Operator     4
Spectre      5
Bulldog      6
Ghost        7
Judge        8
Bucky        9
Sheriff     10
Classic     11
Frenzy      12
Guardian    13
Ares      

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.54      0.14      0.22      4120
           1       0.72      0.95      0.82      9589

    accuracy                           0.71     13709
   macro avg       0.63      0.54      0.52     13709
weighted avg       0.66      0.71      0.64     13709



In [ ]:
t=pd.DataFrame(data=[['Jett','Spectre','130','1','0','0']],columns=['agent_1','gun1_name','damage_round','kills_round','kd_ratio','clutches'])
y_pred = pipe.predict(t)
pipe.predict(t)[0]

# 테스트데이터 넣어보기..  임의의 데이터프레임 1개를 생성 후 예측모델에 적용 후 타겟값 출력.
# flask로 넘길때는 에이전트, 무기만 선택가능하게 하고, 다른 스탯은 평균값으로 defalut 할 예정.'''

0

In [ ]:
# postgre db 연동
import psycopg2

conn = psycopg2.connect(
    host="castor.db.elephantsql.com",
    database="relvoyta",
    user="relvoyta",
    password="HZ8FtlbBcxKe0czIJqOrgligbfHIJrSi")


/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [ ]:
# 데이터보기
cur=conn.cursor()
cur.execute("""SELECT * FROM val_stats_csv""")
print(cur.fetchone())


('NA', 'ShimmyXD', '#NA1', 'Radiant', 135.8, '992', 24.9, 0, 140, 80, 161, '1,506', '1,408', 703, 1.07, 0.7, 29, 208.8, 59, 59.6, 'Fade', 'Viper', 'Omen', 'Vandal', 35, 59, 5, '802', 'Phantom', 33, 62, 5, 220, 'Classic', 36, 60, 3, 147)
